In [1]:
!pip install langchain-text-splitters
!pip install langchain-community
!pip install langchain-openai
!pip install langchain-core
!pip install faiss-cpu
!pip install PyPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 5.4 MB/s eta 0:00:00


In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
import requests
import getpass
import os

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Insira sua chave OpenAI API: ")

Insira sua chave OpenAI API: ··········


In [8]:
pdf_path = 'https://github.com/pedroamelo/cesar-school-topicos-contemporaneos/raw/main/Projeto%20Final%20-%20RAG/Free_Top_SQL_questions_and_answers__1726668375.pdf'
loader = PyPDFLoader(pdf_path)
docs = loader.load()

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(docs)

In [11]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_docs, embeddings)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [12]:
system_template = """Você é um assistente para responder perguntas, use os seguintes trechos de conteúdo para elaborar a resposta. Se não souber a resposta, apenas diga que não sabe. Formule respostas simples de apenas duas frases.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [13]:
# Definição do modelo
llm = ChatOpenAI(model="gpt-4")

# Construir a pipeline RAG
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [14]:
question = "O que é SQL"
response = rag_chain.invoke(question)
print("Resposta:", response)

Resposta: SQL, que significa Structured Query Language, é uma linguagem poderosa usada para acessar, inserir/atualizar/excluir registros e recuperar dados de um banco de dados. Foi projetado especificamente para trabalhar com sistemas de gerenciamento de banco de dados relacional.


In [17]:
question = "Qual a diferença entre uma chave primária e uma chave única?"
response = rag_chain.invoke(question)
print("Resposta:", response)

Resposta: A chave primária não pode ter valor NULL e só pode haver uma por tabela, também por padrão cria um índice clusterizado na coluna. A chave única pode ter um valor NULL e pode haver várias em uma tabela, além disso, cria um índice não clusterizado por padrão.


In [18]:
question = "O que é uma Stored Procedure?"
response = rag_chain.invoke(question)
print("Resposta:", response)

Resposta: Uma Stored Procedure é um grupo nomeado de instruções SQL previamente criadas e armazenadas no banco de dados do servidor. Elas aceitam parâmetros de entrada e são usadas para reduzir o tráfego na rede, melhorar o desempenho e ajudar a garantir a integridade do banco de dados.
